### Q1: 

How many unique species do we observe each month?

In [5]:
import sqlite3
import pandas as pd

In [6]:
conn = sqlite3.connect("data/portal_mammals.sqlite")
c = conn.cursor()

In [38]:
species_month = pd.read_sql_query("""
SELECT 
    DISTINCT month, species_id
FROM 
    surveys
ORDER BY month, species_id
""", conn)

In [40]:
#species_month

In [69]:
species_month_count = pd.read_sql_query("""
WITH species_month AS
(
SELECT 
    DISTINCT month, species_id
FROM 
    surveys
ORDER BY month, species_id
)

SELECT 
    month, 
    count(species_id) as unique_species
FROM
    species_month
GROUP BY
    month
""", conn)

In [101]:
species_month_count

,month,unique_species
0,1,30
1,2,32
2,3,28
3,4,33
4,5,29
5,6,27
6,7,32
7,8,30
8,9,28
9,10,33


### Q2
print the increase or decrease in the number of unique species observed per month 

In [43]:
# method 1: list operation

In [79]:
species_counts = list(species_month_count['unique_species'])

In [84]:
for i in range(len(species_counts)-1):
    print(i+1, species_counts[i+1] - species_counts[i])

1 2
2 -4
3 5
4 -4
5 -2
6 5
7 -2
8 -2
9 5
10 -5
11 0


In [85]:
# method 2: query

In [94]:
delta = pd.read_sql_query("""
WITH species_month AS
(
SELECT 
    DISTINCT month, species_id
FROM 
    surveys
ORDER BY month, species_id
)

SELECT 
    sm1.month, 
    count(sm2.species_id) - count(sm1.species_id) as delta
FROM
    species_month sm1
JOIN
    species_month sm2
ON
    sm1.month + 1 = sm2.month
GROUP BY
    sm1.month
""", conn)

In [93]:
delta

,month,delta
0,1,2
1,2,-4
2,3,5
3,4,-4
4,5,-2
5,6,5
6,7,-2
7,8,-2
8,9,5
9,10,-5


In [104]:
### Method 3 - pandas diff()

In [105]:
# get the difference between adjacent rows in a pandas dataframe
# stack overflow: 
# https://stackoverflow.com/questions/34846146/how-to-calculate-differences-between-consecutive-rows-in-pandas-data-frame

In [108]:
# ok!
species_month_count.diff()

,month,unique_species
0,NaN,NaN
1,1.0,2.0
2,1.0,-4.0
3,1.0,5.0
4,1.0,-4.0
5,1.0,-2.0
6,1.0,5.0
7,1.0,-2.0
8,1.0,-2.0
9,1.0,5.0
